In [ ]:
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import normalize
import numpy as np

In [ ]:
#Some code from NLTK.ORG

from urllib import request #import some library
import nltk, re, pprint
from nltk import word_tokenize
nltk.download('punkt_tab')
nltk.download('stopwords')
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
import scipy


# Function to clean the text
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

# Load the file
file = 'text8.txt'
with open(file, 'r') as f:
    textBad = f.read()

type(textBad) #what is the type of the variable
print(len(textBad)) #what is the length of the text file, number of words

textBad = clean_text(textBad) #clean the text

tokens = word_tokenize(textBad)
type(tokens)
print(len(tokens))
print(tokens[:10])
text = nltk.Text(tokens) #https://www.nltk.org/api/nltk.text.Text.html
print(type(text))
print(text[1024:1062])
print(text.collocations()) #https://www.nltk.org/api/nltk.collocations.html

#Let's build a vocabulary.
words = [w.lower() for w in tokens]
print(type(words))
vocab = sorted(set(words))
print(type(vocab))
print(len(vocab))

#https://www.geeksforgeeks.org/co-occurence-matrix-in-nlp/
#Let's build cooccurrence counts
window_size = 2 #How many words in sequence to consider to be in the window
# Create a list of co-occurring word pairs
co_occurrences = defaultdict(Counter)
for i, word in enumerate(words):
    for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
        if i != j:
            co_occurrences[word][words[j]] += 1

# Create a list of unique words
unique_words = list(set(words))
# Initialize the co-occurrence matrix
co_matrix = np.zeros((len(unique_words), len(unique_words)), dtype=int)

# Populate the co-occurrence matrix
word_index = {word: idx for idx, word in enumerate(unique_words)}
for word, neighbors in co_occurrences.items():
    for neighbor, count in neighbors.items():
        co_matrix[word_index[word]][word_index[neighbor]] = count

# Create a DataFrame for better readability
co_matrix_df = pd.DataFrame(co_matrix, index=unique_words, columns=unique_words)

# Display the co-occurrence matrix
print(co_matrix_df)

#Convert the above matrix to sparse representation, saves memory
print(scipy.sparse.csr_matrix(co_matrix_df))


In [ ]:
from urllib import request  # import some library
import nltk, re, pprint
from nltk import word_tokenize
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix, csr_matrix

# Download NLTK resources
nltk.download('punkt_tab')
nltk.download('stopwords')

# Function to clean the text
def clean_text(text):
    # Convert to lowercase
    text = text.lower()
    # Remove punctuation and digits
    text = re.sub(r'[^\w\s]', '', text)
    # Remove numbers
    text = re.sub(r'\d+', '', text)
    # Remove ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return text

# Load the file
file = 'text8.txt'
with open(file, 'r') as f:
    textBad = f.read()

# Clean the text
textBad = clean_text(textBad)

# Tokenize the text
tokens = word_tokenize(textBad)

# Build vocabulary
words = [w.lower() for w in tokens]
vocab = sorted(set(words))
print(f"Vocabulary size: {len(vocab)}")

# Create a sparse co-occurrence matrix using lil_matrix
window_size = 2  # How many words in sequence to consider in the window
word_index = {word: idx for idx, word in enumerate(vocab)}  # Word-to-index mapping
co_matrix_sparse = lil_matrix((len(vocab), len(vocab)), dtype=int)  # Sparse matrix

# Populate the sparse co-occurrence matrix
for i, word in enumerate(words):
    for j in range(max(0, i - window_size), min(len(words), i + window_size + 1)):
        if i != j:
            co_matrix_sparse[word_index[word], word_index[words[j]]] += 1

# Convert to CSR format for efficiency
co_matrix_sparse = csr_matrix(co_matrix_sparse)
print(f"Sparse co-occurrence matrix shape: {co_matrix_sparse.shape}")
print(f"Non-zero elements: {co_matrix_sparse.nnz}")

# Optionally, convert back to a DataFrame (for small vocabularies only)
# co_matrix_df = pd.DataFrame(co_matrix_sparse.toarray(), index=vocab, columns=vocab)


In [ ]:
print(co_matrix_sparse)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt

# Step 1: Normalize the co-occurrence sparse matrix (row-wise)
normalized_matrix = normalize(co_matrix_sparse, norm='l2', axis=1)

# Step 2: Dimensionality reduction using Truncated SVD
# Reduce dimensions to 100 for computational efficiency
svd = TruncatedSVD(n_components=100, random_state=42)
reduced_matrix = svd.fit_transform(normalized_matrix)

# Step 3: Experiment with different numbers of clusters
k_values = range(2,30)
inertias = []
silhouette_scores = []

for k in k_values:
    print(f"Clustering with k={k}")
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(reduced_matrix)
    labels = kmeans.labels_

    # Inertia (sum of squared distances to centroids)
    inertia = kmeans.inertia_
    # Silhouette score (how well the clusters are separated)
    # silhouette_score_ = silhouette_score(reduced_matrix, labels)

    inertias.append(inertia)
    # silhouette_scores.append(silhouette_score_)

# Step 4: Plot the elbow method and silhouette score
plt.figure(figsize=(12, 6))

# Elbow method plot
plt.subplot(1, 2, 1)
plt.plot(k_values, inertias, marker='o')
plt.title("Elbow Method")
plt.xlabel("Number of clusters")
plt.ylabel("Inertia")

# # Silhouette score plot
# plt.subplot(1, 2, 2)
# # plt.plot(k_values, silhouette_scores, marker='o')
# plt.title("Silhouette Score")
# plt.xlabel("Number of clusters")
# plt.ylabel("Silhouette Score")

plt.tight_layout()
plt.show()


In [ ]:
reduced_matrix.shape

In [ ]:
# K-means clustering
num_clusters = 30
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(reduced_matrix)

# Get cluster labels
labels = kmeans.labels_

In [ ]:
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Reduce the data to 3 dimensions for 3D plotting
svd_3d = TruncatedSVD(n_components=3, random_state=42)
reduced_matrix_3d = svd_3d.fit_transform(reduced_matrix)

# Create a 3D scatter plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Plot each cluster with a different color
for cluster in range(num_clusters):
    cluster_points = reduced_matrix_3d[labels == cluster]  # Filter points by cluster label
    ax.scatter(
        cluster_points[:, 0],  # x-coordinate
        cluster_points[:, 1],  # y-coordinate
        cluster_points[:, 2],  # z-coordinate
        label=f"Cluster {cluster + 1}",
        s=20  # marker size
    )

# Add titles and labels
ax.set_title("3D Visualization of K-means Clusters")
ax.set_xlabel("SVD Dimension 1")
ax.set_ylabel("SVD Dimension 2")
ax.set_zlabel("SVD Dimension 3")

plt.legend()
plt.show()


In [ ]:
import plotly.express as px
import pandas as pd
from sklearn.decomposition import TruncatedSVD
import plotly.io as pio

# Force Plotly to render in the browser
pio.renderers.default = "browser"

# Reduce the data to 3 dimensions for 3D plotting
svd_3d = TruncatedSVD(n_components=3, random_state=42)
reduced_matrix_3d = svd_3d.fit_transform(reduced_matrix)

# Prepare the data for Plotly
df = pd.DataFrame(reduced_matrix_3d, columns=['Dimension 1', 'Dimension 2', 'Dimension 3'])
df['Cluster'] = labels  # Add cluster labels to the DataFrame

# Create an interactive 3D scatter plot
fig = px.scatter_3d(
    df,
    x='Dimension 1',
    y='Dimension 2',
    z='Dimension 3',
    color=df['Cluster'].astype(str),  # Use clusters as colors
    title="Interactive 3D Visualization of K-means Clusters",
    labels={'color': 'Cluster'}
)

# Show the plot
fig.show()
